# Find the monthly retention rate

Find the monthly retention rate of users for each account separately for Dec 2020 and Jan 2021.   
Retention rate is the percentage of active users an account retains (удерживать) over a given period of time.   
In this case, assume the user is retained if he/she stays with the app in any future months.   

For example, if a user was active in Dec 2020 and has activity in any future month, consider them retained for Dec.   
You can assume all accounts are present in Dec 2020 and Jan 2021.   
Your output should have the **account ID and the Jan 2021 retention rate divided by Dec 2020 retention rate**.

In [1]:
import pandas as pd
from pandas import Timestamp

import random

In [2]:
sf_events = pd.DataFrame(
        {
        'date': [Timestamp('2021-01-01 00:00:00'), Timestamp('2021-01-01 00:00:00'), Timestamp('2021-01-06 00:00:00'), Timestamp('2021-01-02 00:00:00'), Timestamp('2020-12-24 00:00:00'), Timestamp('2020-12-08 00:00:00'), Timestamp('2020-12-09 00:00:00'), Timestamp('2021-01-10 00:00:00'), Timestamp('2021-01-11 00:00:00'), Timestamp('2021-01-12 00:00:00'), Timestamp('2021-01-15 00:00:00'), Timestamp('2020-12-17 00:00:00'), Timestamp('2020-12-25 00:00:00'), Timestamp('2020-12-25 00:00:00'), Timestamp('2020-12-25 00:00:00'), Timestamp('2020-12-06 00:00:00'), Timestamp('2020-12-06 00:00:00'), Timestamp('2021-01-14 00:00:00'), Timestamp('2021-02-07 00:00:00'), Timestamp('2021-02-10 00:00:00'), Timestamp('2021-02-01 00:00:00'), Timestamp('2021-02-01 00:00:00'), Timestamp('2020-12-05 00:00:00')],
        'account_id': ['A1', 'A1', 'A1', 'A1', 'A1', 'A1', 'A1', 'A2', 'A2', 'A2', 'A2', 'A2', 'A3', 'A3', 'A3', 'A3', 'A3', 'A3', 'A1', 'A1', 'A2', 'A2', 'A1'],
        'user_id': ['U1', 'U2', 'U3', 'U1', 'U2', 'U1', 'U1', 'U4', 'U4', 'U4', 'U5', 'U4', 'U6', 'U6', 'U6', 'U7', 'U6', 'U6', 'U1', 'U2', 'U4', 'U5', 'U8'],
    }
)

sf_events.info()
sf_events.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        23 non-null     datetime64[ns]
 1   account_id  23 non-null     object        
 2   user_id     23 non-null     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 680.0+ bytes


,date,account_id,user_id
0,2021-01-01,A1,U1
1,2021-01-01,A1,U2
2,2021-01-06,A1,U3


### Pivot table by year and month

In [11]:
monthly_users = pd.DataFrame(sf_events.groupby([pd.Grouper(key='date', freq='Y'), 
                                  pd.Grouper(key='date', freq='M'),
                                  'account_id'])['user_id'].apply(set))
monthly_users

user_id
date       date       account_id              
2020-12-31 2020-12-31 A1          {U2, U1, U8}
                      A2                  {U4}
                      A3              {U6, U7}
2021-12-31 2021-01-31 A1          {U2, U1, U3}
                      A2              {U5, U4}
                      A3                  {U6}
           2021-02-28 A1              {U2, U1}
                      A2              {U5, U4}

In [14]:
monthly_users = pd.DataFrame(sf_events.groupby([pd.Grouper(key='date', freq='Y'), 
                                  pd.Grouper(key='date', freq='M'),
                                  'account_id'])['user_id'].apply(set))

monthly_users.index.names = ['year', 'month', 'account_id']
monthly_users.reset_index(inplace=True)
monthly_users['year'] = monthly_users.year.dt.year
monthly_users['month'] = monthly_users.month.dt.month

monthly_users

,year,month,account_id,user_id
0,2020,12,A1,"{U2, U1, U8}"
1,2020,12,A2,{U4}
2,2020,12,A3,"{U6, U7}"
3,2021,1,A1,"{U2, U1, U3}"
4,2021,1,A2,"{U5, U4}"
5,2021,1,A3,{U6}
6,2021,2,A1,"{U2, U1}"
7,2021,2,A2,"{U5, U4}"


### Filtered pivot table by user_id

In [7]:
for uid in sf_events.user_id.unique():
    temp = monthly_users[monthly_users.user_id.apply(lambda x: True if uid in x else False)]
    print(uid)
    display(temp)

U1


,year,month,account_id,user_id
0,2020,12,A1,"{U2, U1, U8}"
3,2021,1,A1,"{U2, U1, U3}"
6,2021,2,A1,"{U2, U1}"


U2


,year,month,account_id,user_id
0,2020,12,A1,"{U2, U1, U8}"
3,2021,1,A1,"{U2, U1, U3}"
6,2021,2,A1,"{U2, U1}"


U3


...

,year,month,account_id,user_id
2,2020,12,A3,"{U6, U7}"
5,2021,1,A3,{U6}


U7


,year,month,account_id,user_id
2,2020,12,A3,"{U6, U7}"


U8


,year,month,account_id,user_id
0,2020,12,A1,"{U2, U1, U8}"


### Calculate retention rate for each account

In [30]:
monthly_users['retention_rate'] = None

for acc_id in sf_events.account_id.unique():
    
    temp = monthly_users[monthly_users.account_id == acc_id].copy().reset_index(drop=True)
    
    for row in range(temp.shape[0]): # Здесь мы будем брать каждый список с uid
        uid_list = temp.iloc[row]['user_id'] # Список user_id текущего года и текущего месяца
        another_uid = temp.iloc[row+1:].user_id.tolist() # Список user_id всех последующих месяцев
        
        # Метрика = длина рассматриваемого списка uid минус (длина минус список, где остались не пересеченные uid со списками из следующих месяцев)
        # Если простым языком: доля uid, которые встречаются в будущих месяцах
        
        retention_rate = round((len(uid_list) - len(uid_list - set(item for subset in another_uid for item in subset)))  / len(uid_list), 2)
        
        monthly_users.loc[(monthly_users['year'] == temp.iloc[row]['year'])
                           & (monthly_users['month'] == temp.iloc[row]['month']) 
                           & (monthly_users['account_id'] == temp.iloc[row]['account_id']), 'retention_rate'] = retention_rate
        
monthly_users

,year,month,account_id,user_id,retention_rate
0,2020,12,A1,"{U2, U1, U8}",0.67
1,2020,12,A2,{U4},1.0
2,2020,12,A3,"{U6, U7}",0.5
3,2021,1,A1,"{U2, U1, U3}",0.67
4,2021,1,A2,"{U5, U4}",1.0
5,2021,1,A3,{U6},0.0
6,2021,2,A1,"{U2, U1}",0.0
7,2021,2,A2,"{U5, U4}",0.0


In [29]:
result = monthly_users.pivot_table(index='account_id', 
                          columns=['year', 'month'], 
                          values='retention_rate', 
                          aggfunc='sum',
                         sort=False)

# Расчет отношения коэф. удержания января к декабрю
result['ratio_jan_by_dec'] = result[(2021, 1)]/result[(2020, 12)]
result

year        2020  2021      ratio_jan_by_dec
month         12     1    2                 
account_id                                  
A1          0.67  0.67  0.0              1.0
A2          1.00  1.00  0.0              1.0
A3          0.50  0.00  NaN              0.0

![picture_of_bird](task_1_result.png)

# Done!